In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
import os
import json

SERVICE_ACCOUNT_FILE = '../credentials.json'
SCOPES = ['https://www.googleapis.com/auth/drive']
flow = InstalledAppFlow.from_client_secrets_file(SERVICE_ACCOUNT_FILE, SCOPES)
creds = flow.run_local_server(port=8502)
service = build('drive', 'v3', credentials=creds)

KEYFRAME_FOLDER_ID = '1bqJG0CRIIuVIib3pBcA2k8iiRyWlwmq9'

def list_drive_files(folder_id):
    query = f"'{folder_id}' in parents and trashed = false"
    files = []
    try:
        results = service.files().list(q=query, fields="files(id, name, mimeType)").execute()
        files = results.get('files', [])
    except HttpError as error:
        print(f"An error occurred: {error}")
    return files

def get_image_paths(folder_id, video_ID):
    files = list_drive_files(folder_id)
    image_video_dict = {}
    idx = 0
    for file in sorted(files, key=lambda x: x['name']):
        if file['mimeType'] == 'image/jpeg':
            file_id = file['id']
            img_url = f"https://drive.google.com/uc?id={file_id}"
            frame_timestamp = file['name'].replace('.jpg', '')
            image_video_dict[idx] = {'frame_path': img_url, 'video_ID': video_ID, 'timestamp': frame_timestamp}
            idx += 1
    return image_video_dict

def save_json(data, output_folder, filename):
    os.makedirs(output_folder, exist_ok=True)
    
    file_path = os.path.join(output_folder, filename)
    with open(file_path, 'w') as outfile:
        json.dump(data, outfile, indent=4)

def process_keyframe_folders(keyframe_folder_id, output_folder):
    video_folders = list_drive_files(keyframe_folder_id)
    for video_folder in video_folders:
        if video_folder['mimeType'] == 'application/vnd.google-apps.folder': 
            video_ID = video_folder['name'] 
            video_folder_id = video_folder['id']

            subfolders = list_drive_files(video_folder_id)
            for subfolder in subfolders:
                if '_reduced' in subfolder['name']:
                    continue
                
                subfolder_id = subfolder['id']
                image_info_dict = get_image_paths(subfolder_id, video_ID)
                
                filename = f"{subfolder['name']}.json"
                save_json(image_info_dict, output_folder, filename)

output_folder = '../keyframe_information/annotation'
process_keyframe_folders(KEYFRAME_FOLDER_ID, output_folder)

In [ ]:
def combine_json_files(output_folder='../keyframe_information/annotation'):
    
    combined_data = {}
    current_index = 0
    
    try:
        json_files = sorted([f for f in os.listdir(output_folder) if f.startswith('L') and f.endswith('.json')])
        
        for json_file in json_files:
            file_path = os.path.join(output_folder, json_file)
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)

                    for _, value in data.items():
                        combined_data[str(current_index)] = value
                        current_index += 1

                print(f"Processed {json_file}")

            except json.JSONDecodeError:
                print(f"Warning: Error decoding {json_file}")
                continue

        combined_output_path = os.path.join(output_folder, 'combined_data.json')
        with open(combined_output_path, 'w') as f:
            json.dump(combined_data, f, indent=4)

        print(f"\nTotal frames processed: {current_index}")
        print(f"Combined data saved to '{combined_output_path}'")
        
    except FileNotFoundError:
        print(f"Error: Directory '{output_folder}' not found")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

combine_json_files()

NameError: name 'json' is not defined